In [ ]:
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from tensorflow import keras
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Optimizer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras import regularizers
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pickle
import cv2
import os
from PIL import Image
import hickle
from os import listdir 
from os.path import isfile, join, isdir
import random
import sys
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [ ]:
#i3d 5c
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(16,1024)),
    #keras.layers.Dropout(0.5),
    #keras.layers.Dense(512, activation='relu'),
    #keras.layers.Dropout(0.5),
    #keras.layers.Dense(256, activation='relu'),
    #keras.layers.Dropout(0.5),
    keras.layers.Dropout(0.9),
    keras.layers.Dense(64, input_dim=64,kernel_regularizer=regularizers.l2(0.01),activity_regularizer=regularizers.l1(0.01)),    
    keras.layers.Dense(51, activation='softmax')
])
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
#i3d train
data_i3d ='/media/senilab/DATA2/feature_hmdb_i3d_5c/feature_train_rgb_all_i3d_5c'
load_i3d = hickle.load(data_i3d)
data_i3d_label ='/media/senilab/DATA2/feature_hmdb_i3d_5c/label_train_rgb_all_i3d_5c'
load_i3d_label = hickle.load(data_i3d_label)
model.fit(load_i3d, load_i3d_label, epochs=500)

In [ ]:
#i3d 5c test
data_i3d = '/media/senilab/DATA2/feature_hmdb_i3d_5c/feature_rgb_test_i3d_5c'
load_test = hickle.load(data_i3d)
test_feat = load_test['feat']
test_label = load_test['label']
model.evaluate(test_feat,  test_label)

In [ ]:
#feature I3D
train_dir_rgb = '/media/senilab/DATA2/att_feature_5c_16_dp/train_flow/'
list_train_rgb = sorted([f for f in listdir(train_dir_rgb) if isfile(join(train_dir_rgb, f))])

input_feat = np.array([])
input_label = np.array([])

for i in list_train_rgb:
    data_file = train_dir_rgb + i
    data = hickle.load(data_file)
    data_feat = np.array([data['feat']])
    data_label = np.array([np.where(data['label'] == 1)[1][0]])
    
    if input_feat.shape[0] == 0:
        input_feat = data_feat
    else:
        input_feat = np.concatenate([input_feat, data_feat], axis=0)
    
    if input_label.shape[0] == 0:
        input_label = data_label
    else:
        input_label = np.append([input_label], [data_label])

save_file = '/media/senilab/DATA2/feature_attention/feature_flow_train_i3d_5c_sorted'
feature = {}
feature['feat'] = input_feat
feature['label'] = input_label
hickle.dump(feature, save_file)

In [ ]:
#feature I3D
test_dir_rgb = '/media/senilab/DATA2/feature_hmdb_i3d_5c/test_flow/'
list_test_rgb = sorted([f for f in listdir(test_dir_rgb) if isfile(join(test_dir_rgb, f))])

# random.shuffle(list_test_rgb)

test_feat = np.array([])
test_label = np.array([])

for i in list_test_rgb:
    data_file = test_dir_rgb + i
    data_ts = hickle.load(data_file)
    data_feat_ts = np.array([data_ts['feat']])
    data_label_ts = np.array([np.where(data_ts['label'] == 1)[1][0]])
    
    if test_feat.shape[0] == 0:
        test_feat = data_feat_ts
    else:
        test_feat = np.concatenate([test_feat, data_feat_ts], axis=0)
    
    if test_label.shape[0] == 0:
        test_label = data_label_ts
    else:
        test_label = np.append([test_label], [data_label_ts])
        
save_file = '/media/senilab/DATA2/feature_attention/feature_flow_test_i3d_5c_sorted'
feature = {}
feature['feat'] = test_feat
feature['label'] = test_label
hickle.dump(feature, save_file)

In [ ]:
save_file = '/media/senilab/DATA2/feature_attention/feature_rgb_train_i3d_5c'
feature = {}
feature['feat'] = test_feat
feature['label'] = test_label
hickle.dump(feature, save_file)

In [ ]:
data_file = '/media/senilab/DATA2/feature_attention/rgb_train/Veoh_Alpha_Dog_1_walk_f_cm_np1_ba_med_7'
data_tr = hickle.load(data_file)
data_feat_tr = np.array([data_tr['feat'][0]])

In [ ]:
#att
data_att ='/media/senilab/DATA2/feature_attention/rgb_att_i3d_5c'
load_att = hickle.load(data_att)
input_feat_att = load_att['feat']
input_label_att = load_att['label']
model.fit(input_feat_att, input_label_att, epochs=1000)

In [ ]:
#i3d
model.fit(load_i3d, load_i3d_label, epochs=500)